# Introducción a los DataFrames

En este tema veremos:
  - Cómo crear un DataFrame
  - Algunas operaciones básicas sobre DataFrames
      - Mostrar filas
      - Seleccionar columnas
      - Renombrar, añadir y eliminar columnas
      - Eliminar valores nulos y filas duplicadas
      - Reemplazar valores
  - Guardar los DataFrames en diferentes formatos 

## Creación de DataFrames
Un DataFrame puede crearse de distintas formas:

  - A partir de una secuencia de datos
  - A partir de objetos de tipo Row
  - A partir de un RDD o DataSet
  - Leyendo los datos de un fichero
      - Igual que Hadoop, Spark soporta diferentes filesystems: local, HDFS, Amazon S3
          - En general, soporta cualquier fuente de datos que se pueda leer con Hadoop
      - Spark puede acceder a diferentes tipos de ficheros: texto plano, CSV, JSON, [Parquet](https://parquet.apache.org/), [ORC](https://orc.apache.org/), Sequence, etc
        -   Soporta ficheros comprimidos
  - Accediendo a bases de datos relacionales o NoSQL
    -   MySQL, Postgres, etc. mediante JDBC/ODBC
    -   Hive, HBase, Cassandra, MongoDB, AWS Redshift, etc.

In [1]:
import sys

RunningInCOLAB: bool = 'google.colab' in sys.modules

In [2]:
if RunningInCOLAB:
    !sudo apt-get update ; sudo apt-get install -y default-jre-headless
    %env JAVA_HOME=/usr/lib/jvm/default-java

In [ ]:
%pip install pyspark

## Creando DataFrames a partir de una secuencia o lista de datos

In [ ]:
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession

# Elegir el máster de Spark dependiendo de si se ha definido la variable de entorno HADOOP_CONF_DIR o YARN_CONF_DIR
SPARK_MASTER: str = (
    "yarn" if "HADOOP_CONF_DIR" in os.environ or "YARN_CONF_DIR" in os.environ else "local[*]"
)

# Creamos un objeto SparkSession (o lo obtenemos si ya está creado)
spark: SparkSession = (
    SparkSession.builder.appName("Mi aplicacion TCDM")
    .config("spark.rdd.compress", "true")
    .config("spark.executor.memory", "3g")
    .config("spark.driver.memory", "3g")
    .master(SPARK_MASTER)
    .getOrCreate()
)

sc: SparkContext = spark.sparkContext

In [ ]:
from pprint import pp

pp(sc._conf.getAll())

In [ ]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

# Creando un DataFrame desde un rango y añadiéndole dos columnas
df: DataFrame = spark.range(1, 7, 2).toDF("n")
df.show()

# Añadiendo dos columnas al DataFrame
# La expresión para la columna puede incluir operadores.
df.withColumn("n1", col("n") + 1).withColumn("n2", 2 * col("n1")).show()

## Creando DataFrames con esquema
A la hora de crear un DataFrame, es conveniente especificar el esquema del mismo:

  - El esquema define los nombres y tipos de datos de las columnas.
  - Se usa un objeto de tipo `StructType` para definir el nombre y tipo de las columnas, y un objeto de tipo `StructField` para definir el nombre y tipo de una columna.
  - Los tipos de datos que utiliza Spark están definidos en:
      - Para PySpark: https://spark.apache.org/docs/latest/sql-ref-datatypes.html.

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import DataType, FloatType, StringType, StructField, StructType

# Definimos el esquema del DataFrame
esquemaNotas: DataType = StructType(
    fields=[
        StructField(name="nombre", dataType=StringType(), nullable=False),
        StructField(name="nota", dataType=FloatType(), nullable=True),
        StructField(name="calificación", dataType=StringType(), nullable=True),
    ]
)

# Crear Row con campos nombrados explícitamente
filas: list[Row] = [
    Row(nombre="Pepe", nota=5.1, calificación="Aprobado"),
    Row(nombre="Juan", nota=4.0, calificación="Suspenso"),
    Row(nombre="Manuel", nota=None, calificación=None),
]

dfNotas: DataFrame = spark.createDataFrame(filas, schema=esquemaNotas)
dfNotas.show()
dfNotas.printSchema()

## Creando DataFrames a partir de un fichero de texto

Cada línea del fichero se guarda como una fila (e incluso detecta automáticamente si el fichero está comprimido):

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/dsevilla/tcdm-public/refs/heads/25-26/datos/quijote.txt.gz",
    "quijote.txt.gz",
)

In [ ]:
# OJO: Se supone que el usuario que se usa es "luser" y que tiene permisos para escribir en el directorio /user/luser en HDFS.
if SPARK_MASTER == "yarn":
    !hdfs dfs -put quijote.txt.gz /user/luser/

**Nota sobre YARN y HDFS**:

En modo cluster (YARN), los archivos deben estar en **HDFS** (Hadoop Distributed File System) para que todos los workers del cluster puedan accederlos.

- `/user/luser/` es el directorio home del usuario `luser` en HDFS
- Debe existir y tener permisos de escritura: `hdfs dfs -mkdir -p /user/luser`
- En modo `local[*]` no es necesario, Spark lee del filesystem local

In [ ]:
dfQuijote: DataFrame = spark.read.text("quijote.txt.gz")
dfQuijote.show(50, truncate=False)

## Creando DataFrames a partir de un fichero CSV

Como ejemplo vamos a utilizar el fichero de preguntas y respuestas de Stack Overflow en Español, que hemos utilizado en otras ocasiones. Es un fichero CSV, con unos campos que son:

- `Id`: integer: La identificación de la pregunta o respuesta
- `AcceptedAnswerId`: integer: La identificación de la respuesta aceptada (si existe)
- `AnswerCount`: integer: El número de respuestas
- `Body`: string: El cuerpo de la pregunta o respuesta
- `ClosedDate`: timestamp: Fecha de cierre de la pregunta (si está cerrada)
- `CommentCount`: integer: Número de comentarios
- `CommunityOwnedDate`: timestamp: (no se usará)  
- `ContentLicense`: string: Licencia de contenido
- `CreationDate`: timestamp: La fecha de creación
- `FavoriteCount`: integer: Número de favoritos
- `LastActivityDate`: timestamp: (no se usará)
- `LastEditDate`: timestamp: (no se usará)
- `LastEditorDisplayName`: string: (no se usará)
- `LastEditorUserId`: integer: (no se usará)
- `OwnerDisplayName`: string: El nombre del propietario (si se borró el usuario) 
- `OwnerUserId`: integer: El identificador del propietario
- `ParentId`: integer: El identificador de la pregunta padre (si es una respuesta)
- `PostTypeId`: integer: El tipo de post (1 = pregunta, 2 = respuesta, etc.)
- `Score`: integer: La puntuación de la pregunta o respuesta
- `Tags`: string: El conjunto de etiquetas
- `Title`: string: El título de la pregunta
- `ViewCount`: integer: El número de visitas

Los campos se encuentran separados por el símbolo `","`, y el carácter de escape de comillas es el propio carácter de comillas.

### Leemos el fichero infiriendo el esquema

In [ ]:
import tarfile
import urllib.request
from pathlib import Path
from tarfile import TarInfo

# URLs de descarga
file: str = "es.stackoverflow.csv.tar.gz"
URL: str = f"https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/{file}"

# Descargar el fichero tar.gz
urllib.request.urlretrieve(URL, file)
if not Path(file).exists():
    raise FileNotFoundError("No se pudieron descargar los datos de StackOverflow")

# Extraer el fichero Posts.csv del tar.gz
with tarfile.open(file, "r:gz") as tar:
    member: TarInfo | None = next(filter(lambda m: m.name == "Posts.csv", tar.getmembers()), None)
    if member is not None:
        try:
            tar.extract(member=member, filter='data')
        except TypeError:
            # Compatibilidad con versiones anteriores sin parámetro filter
            tar.extractall(members=[member])

assert Path("Posts.csv").exists(), "No se pudo extraer Posts.csv del fichero tar.gz"

In [ ]:
# OJO: Se supone que el usuario que se usa es "luser" y que tiene permisos para escribir en el directorio /user/luser en HDFS.
if SPARK_MASTER == "yarn":
    !hdfs dfs -put Posts.csv /user/luser/

In [ ]:
dfSEInfered: DataFrame = (
    spark.read.format("csv")
    .option("mode", "FAILFAST")
    .option("sep", ",")
    .option("escape", '"')
    .option("inferSchema", "true")
    .option("lineSep", "\r\n")
    .option("header", "true")
    .option("nullValue", "")
    .load("Posts.csv")
)

Algunas opciones:

1. ``mode``: especifica qué hacer cuando se encuentra registros corruptos
    - ``PERMISSIVE``: pone todos los campos a null cuando se encuentra un registro corrupto (valor por defecto)
    - ``DROPMALFORMED``: elimina las filas con registros corruptos
    - ``FAILFAST``: da un error cuando se encuentra un registro corrupto
2. ``sep``: separador entre campos (por defecto ",")
3. ``inferSchema``: especifica si se deben inferir el tipo de las columnas (por defecto "false")
3. ``lineSep``: separador de líneas (por defecto "\n"). Lo hemos cambiado a "\r\n" porque el fichero se ha creado en Windows, aunque da un warning, funciona correctamente
4. ``header``: si "true" se toma la primera fila como cabecera (por defecto "false")
5. ``nullValue``: carácter o cadena que representa un NULL en el fichero (por defecto "")
6. ``compression``: tipo de compresión utilizada (por defecto "none")
  
Las opciones son similares para otros tipos de ficheros.

In [ ]:
# Vemos 5 filas
dfSEInfered.show(5)

In [ ]:
# Vemos como se ha inferido el esquema
dfSEInfered.schema

In [ ]:
# Otra forma de verlo
dfSEInfered.printSchema()

### Leemos especificando el esquema

El esquema inferido tiene ciertos fallos, como considerar algunos campos como strings cuando deberían ser enteros, o tipos Timestamp en vez de Date. Por ello, vamos a especificar el esquema.

In [ ]:
from pyspark.sql.types import (
    DataType,
    IntegerType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)

# Defino el esquema para los elementos de la tabla
# StructType -> Permite definir un esquema para el DF a partir de una lista de StructFields
# StructField -> Definen el nombre y tipo de cada columna, así como si es nullable o no (campo True)
dfSE_Schema: DataType = StructType(
    [
        StructField("Id", IntegerType(), False),
        StructField("AcceptedAnswerId", IntegerType(), True),
        StructField("AnswerCount", IntegerType(), True),
        StructField("Body", StringType(), True),
        StructField("ClosedDate", TimestampType(), True),
        StructField("CommentCount", IntegerType(), True),
        StructField("CommunityOwnedDate", TimestampType(), True),
        StructField("ContentLicense", StringType(), True),
        StructField("CreationDate", TimestampType(), True),
        StructField("FavoriteCount", IntegerType(), True),
        StructField("LastActivityDate", TimestampType(), True),
        StructField("LastEditDate", TimestampType(), True),
        StructField("LastEditorDisplayName", StringType(), True),
        StructField("LastEditorUserId", IntegerType(), True),
        StructField("OwnerDisplayName", StringType(), True),
        StructField("OwnerUserId", IntegerType(), True),
        StructField("ParentId", IntegerType(), True),
        StructField("PostTypeId", IntegerType(), True),
        StructField("Score", IntegerType(), True),
        StructField("Tags", StringType(), True),
        StructField("Title", StringType(), True),
        StructField("ViewCount", IntegerType(), True),
    ]
)

# Creo el DataFrame con el esquema definido
dfSE: DataFrame = (
    spark.read.format("csv")
    .option("mode", "FAILFAST")
    .option("inferSchema", "false")
    .option("sep", ",")
    .option("header", "true")
    .option("nullValue", "")
    .option("lineSep", "\r\n")
    .option("escape", '"')
    .schema(dfSE_Schema)
    .load("Posts.csv")
)

## Trabajo con ficheros Parquet

Al igual que hemos hecho en esta asignatura y en otras, trabajaremos con el fichero `.parquet`. Hemos visto que se puede especificar el esquema de un fichero CSV, pero como vimos el fichero Parquet ya lo lleva implícito. Probémoslo:

In [ ]:
import urllib.request
from pathlib import Path

from pyspark.sql.dataframe import DataFrame

# Descargar el fichero Parquet si no existe
# (esto se debe hacer porque Spark no puede leer directamente desde URLs HTTPS)
parquet_file = "Posts.parquet"
if not Path(parquet_file).exists():
    URL_PARQUET = (
        "https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/Posts.parquet"
    )
    print(f"Descargando {parquet_file}...")
    urllib.request.urlretrieve(URL_PARQUET, parquet_file)
    print("Descarga completada.")

# Leer el fichero Parquet desde el filesystem local
dfSE: DataFrame = spark.read.format('parquet').option('mergeSchema', 'true').load(parquet_file)

dfSE.printSchema()

In [ ]:
dfSE.cache()

**¿Por qué cachear el DataFrame?**

Vamos a realizar múltiples operaciones sobre `dfSE` (mostrar, seleccionar, filtrar, etc.), por lo que lo mantenemos en memoria con `.cache()` para evitar recalcularlo desde el archivo CSV cada vez que lo usemos.

**Cuándo usar cache()**:
- Cuando vas a reutilizar el mismo DataFrame en múltiples acciones
- Después de transformaciones costosas (joins, agregaciones)
- **No cachear** DataFrames muy grandes que no caben en memoria

In [ ]:
dfSE.sort("Id").show()

In [ ]:
dfSE.printSchema()

# Operaciones básicas con DataFrames


### Mostrar filas

In [ ]:
# show(n) permite mostrar las primeras n filas (por defecto, n=20)
dfSE.show(5)

In [ ]:
# Podemos indicar que no trunque los campos largos
dfSE.show(5, truncate=False)

In [ ]:
from pyspark.sql import Row

# take(n) devuelve las n primeras filas como una lista Python de objetos Row
lista: list[Row] = dfSE.take(5)
pp(lista[1])
# collect() devuelve todo el DataFrame como una lista Python de objetos Row
# Si el DataFrame es muy grande podría colapsar al Driver
# lista2 = dfSE.collect()
# print(lista2[10])

In [ ]:
# sample(withReplacement, fraction, seed=None) devuelve un nuevo Dataframe con una fracción de las filas
dfSESampled: DataFrame = dfSE.sample(False, 0.1, seed=None)
print(f"N de filas original = {dfSE.count()}; n de filas muestreadas = {dfSESampled.count()}")

In [ ]:
# limit(n) limita a n el número de filas obtenidas
dfSE_10filas: DataFrame = dfSE.sample(False, 0.1, seed=None).limit(10)
print(f"N de filas muestreadas = {dfSE_10filas.count()}")
dfSE_10filas.show()

### Ejecutar una operación sobre cada una de las filas
El método `foreach` aplica una función a cada una de las filas

- El DataFrame no se modifica y no se crea ningún otro DataFrame
- El `foreach`se ejecuta en los workers

In [ ]:
from pyspark.sql import Row


def printid(f: Row) -> None:
    print(f["Id"])


dfSE_10filas.foreach(printid)

**Importante sobre `foreach()`**:

La función que pasas a `foreach()` se ejecuta en los **workers remotos**, no en el driver donde corre el notebook. Por eso **NO verás el output** de los `print()` en la consola (en el caso de ejecutar en *clúster*).

Para ver resultados:
1. Usa `.take()` o `.collect()` primero para traer datos al driver
2. O usa `foreach()` para efectos secundarios (escribir a BD, enviar a API, etc.)

### Seleccionar columnas

In [ ]:
# Crea un nuevo DataFrame seleccionando columnas por nombre
dfIdBody: DataFrame = dfSE.select("Id", "Body")
dfIdBody.show(5)

print(f"El objeto dfIdCuerpo es de tipo {type(dfIdBody)}.")

In [ ]:
# Otra forma de indicar a las columnas
dfIdBody2: DataFrame = dfSE.select(dfSE.Id, dfSE.Body)
dfIdBody2.show(5)

In [ ]:
# También es posible indicar objetos de tipo Column
from pyspark.sql.column import Column
from pyspark.sql.functions import col

colId: Column = col("Id")
colCreaDate: Column = col("CreationDate")
print(f"El objeto colId es de tipo {type(colId)}.")
print(f"El objeto colCreaDate es de tipo {type(colCreaDate)}.")

In [ ]:
# Y crear un DataFrame a partir de objetos Column, renombrando columnas
dfIdFechaCuerpo: DataFrame = dfSE.select(
    colId, colCreaDate.alias("Fecha_Creación"), dfSE.Body.alias("Cuerpo")
)
dfIdFechaCuerpo.show(5)

In [ ]:
from pyspark.sql.functions import expr

# El DataFrame anterior usando expresiones
dfIdFechaCuerpoExpr: DataFrame = dfSE.select(
    expr("Id AS ID"), expr("CreationDate AS `Fecha_Creación`"), expr("Body AS Cuerpo")
)
dfIdFechaCuerpoExpr.show(5)

In [ ]:
# Se pueden usar expresiones más complejas
dfSE.selectExpr(
    "*", "(AnswerCount IS NOT NULL) as respuestaValida"  # Selecciona todas las columnas
).show()

### Renombrar, añadir y eliminar columnas


In [ ]:
# Renombramos la columna CreationDate
# NOTA: withColumnRenamed() NO modifica dfSE, crea un NUEVO DataFrame
# Los DataFrames son inmutables, pero aquí reasignamos la variable dfSE
dfSE_renamed: DataFrame = dfSE.withColumnRenamed("CreationDate", "Fecha_de_creación")
dfSE_renamed.select(
    "Fecha_de_creación", dfSE_renamed.ViewCount.alias("Número_de_vistas"), "Score", "PostTypeId"
).show(truncate=False)

In [ ]:
# Añadimos una nueva columna con todos sus valores iguales a 1
from pyspark.sql.functions import lit

# lit() convierte un literal de Python al formato interno de Spark (IntegerType en este caso)
# withColumn() crea un NUEVO DataFrame, no modifica el original (inmutabilidad)
dfSE_with_ones: DataFrame = dfSE_renamed.withColumn("unos", lit(1))
dfSE_with_ones.show(5)

In [ ]:
# Elimina una columna con drop
# drop() también crea un NUEVO DataFrame (inmutabilidad)
dfSE_clean: DataFrame = dfSE_with_ones.drop(col("unos"))
dfSE_clean.columns

**Concepto clave: Inmutabilidad de DataFrames**

Observa que cada operación (`withColumnRenamed`, `withColumn` y `drop`) **crea un nuevo DataFrame**. 

Los DataFrames son **inmutables** - nunca se modifican en su lugar. Esto permite:
- Optimizaciones del motor Spark
- Seguridad en entornos distribuidos
- Posibilidad de cachear y reutilizar DataFrames sin efectos secundarios

En el resto del notebook, para simplificar, reasignaremos la variable `dfSE`, pero recuerda que técnicamente son DataFrames distintos.

### Eliminar valores nulos y duplicados

**Diferencia entre `"any"` y `"all"`**:

- `dropna("any")`: Elimina la fila si **al menos una columna** es NULL (más restrictivo)
- `dropna("all")`: Elimina la fila solo si **todas las columnas** son NULL (menos restrictivo)

Con `subset`:
- `dropna("any", subset=[...])`: Elimina si **alguna de las columnas especificadas** es NULL
- `dropna("all", subset=[...])`: Elimina solo si **todas las columnas especificadas** son NULL

In [ ]:
# Eliminamos todas las filas que tengan null en alguna de sus columnas
dfNoNulls: DataFrame = dfSE.dropna("any")
print(f"Numero de filas inicial: {dfSE.count()}; número de filas sin null: {dfNoNulls.count()}.")

In [ ]:
# Elimina las filas que tengan null en todas sus columnas
dfNingunNull: DataFrame = dfSE.dropna("all")
print(f"Número de filas con todo a null: {dfSE.count() - dfNingunNull.count()}.")

In [ ]:
# Elimina las filas duplicadas
dfSinDuplicadas: DataFrame = dfSE.dropDuplicates()
print(f"Número de filas duplicadas: {dfSE.count() - dfSinDuplicadas.count()}.")

In [ ]:
# Elimina las filas duplicadas en alguna columna
dfSinUserDuplicado: DataFrame = dfSE.dropDuplicates(["OwnerUserId"])
print(f"Número de usuarios únicos: {dfSinUserDuplicado.count()}.")

In [ ]:
# Ejemplos con subset de columnas

# Elimina filas donde ViewCount O AcceptedAnswerId son null (al menos una)
dfNoNullViewCountAcceptedAnswerId: DataFrame = dfSE.dropna(
    "any", subset=["ViewCount", "AcceptedAnswerId"]
)
print(
    "Número de filas con ViewCount y AcceptedAnswerId ambos no nulos: {}.".format(
        dfNoNullViewCountAcceptedAnswerId.count()
    )
)

# Elimina filas donde ViewCount y AcceptedAnswerId son AMBOS null
dfAtLeastOneNotNull: DataFrame = dfSE.dropna("all", subset=["ViewCount", "AcceptedAnswerId"])
print(
    "Número de filas con ViewCount O AcceptedAnswerId al menos uno no nulo: {}.".format(
        dfAtLeastOneNotNull.count()
    )
)

### Reemplazar valores

In [ ]:
# Reemplazamos los null en los campos ViewCount y AnswerCount
dfSE: DataFrame = dfSE.fillna(0, subset=["ViewCount", "AnswerCount"])
dfSE.show(5)

In [ ]:
# Ejemplo más realista: Normalizar valores de PostTypeId
# 1 = "Question", 2 = "Answer"
# (se podría utilizar replace si la conversión de valores fuera del mismo tipo)

from pyspark.sql.functions import when

print("Antes de la transformación:")
dfSE.select("Id", "PostTypeId").show(10)

print("\nDespués de la transformación (1→'Question', 2→'Answer'):")
dfSE_normalized: DataFrame = dfSE.withColumn(
    "PostTypeId",
    when(col("PostTypeId") == 1, "Question")
    .when(col("PostTypeId") == 2, "Answer")
    .otherwise(col("PostTypeId").cast(StringType())),
)
dfSE_normalized.select("Id", "PostTypeId").show(10)

# Guardando DataFrames

Al igual que con la lectura, Spark puede guardar los DataFrames en múltiples formatos

- CSV, JSON, Parquet, Hadoop...

También puede escribir en bases de datos

**Modos de escritura**:

Al guardar DataFrames, puedes especificar el modo con `.mode()`:

- `"overwrite"`: Sobrescribe el directorio si ya existe (cuidado: borra los datos anteriores)
- `"append"`: Añade los datos al final sin borrar lo existente
- `"ignore"`: No hace nada si el directorio ya existe (silencioso)
- `"error"` o `"errorifexists"` (default): Lanza error si ya existe

**Comparación de formatos**:

| Formato | Ventajas | Desventajas | Cuándo usar |
|---------|----------|-------------|-------------|
| **CSV** | Legible por humanos, universal | Grande, sin esquema tipado, lento | Intercambio simple, datos pequeños |
| **JSON** | Flexible, soporta anidación | Grande, más lento | APIs, logs, datos semi-estructurados |
| **Parquet** | Comprimido, columnar, muy rápido, preserva tipos | No legible directamente | **Big Data** (recomendado) |

In [ ]:
# Guardo el DataFrame dfSE en formato JSON
dfSE.write.format("json").mode("overwrite").save("dfSE.json")

In [ ]:
%%sh
ls -lh dfSE.json
head dfSE.json/part-*.json

In [ ]:
# Guardo el DataFrame usando Parquet
dfSE.write.format("parquet").mode("overwrite").option("compression", "gzip").save("dfSE.parquet")

In [ ]:
print(dfSE.rdd.getNumPartitions())

In [ ]:
%%sh
# Parquet usa por defecto formato comprimido snappy
ls -lh dfSE.parquet

Se crean tantos ficheros como particiones tenga el DataFrame.

In [ ]:
dfSE2: DataFrame = dfSE.repartition(2)
# Guardo el DataFrame  usando Parquet, con compresión gzip
dfSE2.write.format("parquet").mode("overwrite").option("compression", "gzip").save("dfSE2.parquet")

In [ ]:
%%sh
ls -lh dfSE2.parquet

#### Particionado
Permite particionar los ficheros guardados por el valor de una columna

- Se crea un directorio por cada valor diferente en la columna de particionado
    - Todos los datos asociados a ese valor se guardan en ese directorio
- Permite simplificar el acceso a los valores asociados a una clave


In [ ]:
# Guardo el DataFrame particionado por el PostTypeId (usando Parquet)
dfSE.write.format("parquet").mode("overwrite").partitionBy("PostTypeId").save(
    "dfSE-particionado.parquet"
)

In [ ]:
%%sh
ls dfSE-particionado.parquet
ls -lh dfSE-particionado.parquet/PostTypeId=2
rm -rf dfSE-particionado.parquet

## Recordatorio: Lazy Evaluation

La mayoría de operaciones que hemos visto son **transformaciones lazy** (perezosas):
- `.select()`, `.withColumn()`, `.drop()`, `.filter()`, `.dropna()`, etc.

**No se ejecutan inmediatamente** - Spark solo construye un plan de ejecución.

Las **acciones** que disparan la ejecución son:
- `.show()`, `.count()`, `.collect()`, `.take()`
- `.write.save()` (guardar a disco)

Esto permite que Spark optimice todo el pipeline antes de ejecutar.

---

## Errores Comunes

### 1. **`Py4JJavaError` o Java no encontrado**
- **Causa**: Spark no encuentra Java o la versión es incompatible (necesita Java 8, 11 o 17)
- **Solución**: Instalar Java 17 y configurar `JAVA_HOME`

### 2. **`AnalysisException: Path does not exist`**
- **Causa**: En modo YARN, el archivo no está en HDFS
- **Solución**: Copiar con `hdfs dfs -put archivo.csv /user/luser/`

### 3. **Out of Memory**
- **Causa**: Cachear DataFrames muy grandes o hacer `.collect()` de millones de filas
- **Solución**: Usar `.take(n)` o `.sample()`, no cachear innecesariamente

### 4. **Inferencia de esquema lenta**
- **Causa**: `inferSchema=true` debe leer todo el archivo
- **Solución**: Especificar el esquema explícitamente con `StructType`

### 5. **Variables no actualizadas después de transformaciones**
- **Causa**: Olvidar reasignar la variable (los DataFrames son inmutables)
- **Solución**: `df = df.withColumn(...)` o usar nombres distintos